In [1]:
# ------------ IMPORTS ------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os # OS ko import karein directory banane ke liye

# ------------ 1. SETUP PATHS & CREATE DIRECTORY (FIXED) ------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Yahan humne DATA_DIR ka poora path diya hai (Jaisa pehle fix kiya tha)
DATA_DIR = r'C:\JN\Aerial_Object_Classification\classification_dataset'

# Yahan humne 'models' directory ka path set kiya hai
MODELS_DIR = r'C:\JN\Aerial_Object_Classification\models'
BEST_MODEL_PATH = os.path.join(MODELS_DIR, 'best_efficientnet_model.h5')

# *** YAHAN ERROR FIX KIYA GAYA HAI ***
# Model training se pehle 'models' folder ko banayein
# exist_ok=True ka matlab hai ki agar folder pehle se hai, to error na de
os.makedirs(MODELS_DIR, exist_ok=True)
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")


# ------------ 2. DATA LOADING & PREPROCESSING ------------
print("\nLoading and preprocessing data...")

# Training data ke liye augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation aur Test data ke liye sirf normalization
test_datagen = ImageDataGenerator(rescale=1./255)

# Data Loaders
try:
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TRAIN'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    validation_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'VALID'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TEST'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    print("Data loaded successfully.")
    print("Classes found:", train_generator.class_indices)

except FileNotFoundError as e:
    print(f"ERROR: Data directory not found. Please check the path: {e}")
    # Agar data nahi milta hai to code ko yahin rok dein
    raise e


# ------------ 3. MODEL BUILDING & TRAINING ------------
print("\nBuilding model (EfficientNetB0)...")

# Base model load karna
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False # Base model ko freeze karna

# Model structure
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # Binary classification
])

# Model compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks set karna
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Model Checkpoint (Ab ye 'models' folder mein save karega)
model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_PATH, # Fixed path
    monitor='val_accuracy',
    save_best_only=True
)

print("\nStarting model training...")
# Model train karna
history = model.fit(
    train_generator,
    epochs=50, # Aap epochs kam ya zyada kar sakte hain
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)
print("Model training finished.")


# ------------ 4. MODEL EVALUATION ------------
print("\nEvaluating model on TEST set...")

# Best model ko load karna (jo ModelCheckpoint ne save kiya tha)
try:
    best_model = tf.keras.models.load_model(BEST_MODEL_PATH)

    # Test set par evaluate karna
    test_loss, test_acc = best_model.evaluate(test_generator)
    print(f"\nTest Accuracy: {test_acc*100:.2f}%")
    print(f"Test Loss: {test_loss:.4f}")

    # Predictions
    predictions = best_model.predict(test_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = test_generator.classes

    # Class ke naam (generator se)
    class_labels = list(test_generator.class_indices.keys())

    # Classification Report
    print("\n--- Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    # Confusion Matrix
    print("\n--- Confusion Matrix ---")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

except FileNotFoundError:
    print(f"ERROR: Trained model not found at {BEST_MODEL_PATH}. Training might have failed.")

print("\n--- Notebook execution complete ---")

Data directory: C:\JN\Aerial_Object_Classification\classification_dataset
Models directory: C:\JN\Aerial_Object_Classification\models

Loading and preprocessing data...
Found 2662 images belonging to 2 classes.
Found 442 images belonging to 2 classes.
Found 215 images belonging to 2 classes.
Data loaded successfully.
Classes found: {'bird': 0, 'drone': 1}

Building model (EfficientNetB0)...

Starting model training...
Epoch 1/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5096 - loss: 0.6956

84/84 ━━━━━━━━━━━━━━━━━━━━ 445s 5s/step - accuracy: 0.5143 - loss: 0.6961 - val_accuracy: 0.4910 - val_loss: 0.7025
Epoch 2/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5004 - loss: 0.7000

84/84 ━━━━━━━━━━━━━━━━━━━━ 284s 3s/step - accuracy: 0.5094 - loss: 0.6987 - val_accuracy: 0.5090 - val_loss: 0.6929
Epoch 3/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 240s 3s/step - accuracy: 0.5184 - loss: 0.6976 - val_accuracy: 0.4910 - val_loss: 0.7047
Epoch 4/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.5210 - loss: 0.6948 - val_accuracy: 0.4910 - val_loss: 0.6936
Epoch 5/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 236s 3s/step - accuracy: 0.5086 - loss: 0.6957 - val_accuracy: 0.5090 - val_loss: 0.6928
Epoch 6/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 232s 3s/step - accuracy: 0.5038 - loss: 0.6974 - val_accuracy: 0.4910 - val_loss: 0.6991
Epoch 7/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 241s 3s/step - accuracy: 0.5038 - loss: 0.7004 - val_accuracy: 0.5090 - val_loss: 0.6925
Epoch 8/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 223s 3s/step - accuracy: 0.5244 - loss: 0.6947 - val_accuracy: 0.5090 - val_loss: 0.6948
Epoch 9/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5167 - loss: 0.7018

84/84 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.5162 - loss: 0.7023 - val_accuracy: 0.5543 - val_loss: 0.6926
Epoch 10/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.5177 - loss: 0.6997 - val_accuracy: 0.4910 - val_loss: 0.7068
Epoch 11/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.5237 - loss: 0.6973 - val_accuracy: 0.5090 - val_loss: 0.6957
Epoch 12/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.5041 - loss: 0.7005 - val_accuracy: 0.5090 - val_loss: 0.6923
Epoch 13/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.5056 - loss: 0.7009 - val_accuracy: 0.4910 - val_loss: 0.7019
Epoch 14/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.5177 - loss: 0.6957 - val_accuracy: 0.5090 - val_loss: 0.6975
Epoch 15/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.5026 - loss: 0.6993 - val_accuracy: 0.4910 - val_loss: 0.6933
Epoch 16/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.5075 - loss: 0.6962 - val_accuracy: 0.4910 - val_l

7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5628 - loss: 0.6899

Test Accuracy: 56.28%
Test Loss: 0.6899
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step

--- Classification Report ---
              precision    recall  f1-score   support

        bird       0.56      1.00      0.72       121
       drone       0.00      0.00      0.00        94

    accuracy                           0.56       215
   macro avg       0.28      0.50      0.36       215
weighted avg       0.32      0.56      0.41       215


--- Confusion Matrix ---
[[121   0]
 [ 94   0]]

--- Notebook execution complete ---


C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [1]:
# ------------ IMPORTS ------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

print("--- Starting Complete Training & Fine-Tuning Process ---")

# ------------ 1. SETUP PATHS & CREATE DIRECTORY ------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

DATA_DIR = r'C:\JN\Aerial_Object_Classification\classification_dataset'
MODELS_DIR = r'C:\JN\Aerial_Object_Classification\models'
# Hum naya naam istemal karenge taaki purani corrupt file se conflict na ho
BEST_MODEL_PATH = os.path.join(MODELS_DIR, 'final_finetuned_model.h5')

# 'models' directory banayein
os.makedirs(MODELS_DIR, exist_ok=True)
print(f"Data directory: {DATA_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"Best model will be saved to: {BEST_MODEL_PATH}")

# ------------ 2. DATA LOADING & PREPROCESSING ------------
print("\nLoading and preprocessing data...")

# Training data ke liye augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation aur Test data ke liye sirf normalization
test_datagen = ImageDataGenerator(rescale=1./255)

try:
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TRAIN'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )

    validation_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'VALID'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TEST'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    print("Data loaded successfully.")
    print("Classes found:", train_generator.class_indices)

except FileNotFoundError as e:
    print(f"ERROR: Data directory not found. Please check the path: {e}")
    raise e

# ------------ 3. MODEL BUILDING ------------
print("\nBuilding model (EfficientNetB0)...")

# Base model load karna
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
# Shuru mein base model ko freeze rakhein
base_model.trainable = False 

# Model structure
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation='sigmoid') # Binary classification
])

# ------------ 4. PHASE 1: FEATURE EXTRACTION TRAINING ------------
# Is phase mein hum sirf top layer (classifier) ko train karenge
print("\n--- PHASE 1: Starting Feature Extraction (Training Classifier) ---")

model.compile(
    optimizer=Adam(learning_rate=1e-3), # Standard learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Hum yahan sirf 10 epochs ke liye train karenge
PHASE_1_EPOCHS = 10
history_phase1 = model.fit(
    train_generator,
    epochs=PHASE_1_EPOCHS,
    validation_data=validation_generator
)

print("\n--- PHASE 1: Feature Extraction Complete ---")

# ------------ 5. PHASE 2: FINE-TUNING TRAINING ------------
print("\n--- PHASE 2: Starting Fine-Tuning (Training Base Model) ---")

# Base model ko 'trainable' banayein
base_model.trainable = True

# Aakhri 30 layers ko unfreeze karein
unfreeze_layers = 30
for layer in base_model.layers[-unfreeze_layers:]:
    layer.trainable = True

print(f"Unfreezing last {unfreeze_layers} layers of the base model.")

# Model ko bahut kam learning rate ke sath Re-compile karein
model.compile(
    optimizer=Adam(learning_rate=1e-5), # Bahut kam learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks set karein (Is baar humara pickle error fix shamil hai)
model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_PATH,
    monitor='val_accuracy',
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5 # restore_best_weights=True ko hata diya gaya hai
)

# Fine-tuning ke liye 20 aur epochs train karein
PHASE_2_EPOCHS = 20
TOTAL_EPOCHS = PHASE_1_EPOCHS + PHASE_2_EPOCHS

history_phase2 = model.fit(
    train_generator,
    epochs=TOTAL_EPOCHS,
    initial_epoch=PHASE_1_EPOCHS, # Training ko 10th epoch se aage badhayein
    validation_data=validation_generator,
    callbacks=[model_checkpoint, early_stopping]
)

print("\n--- PHASE 2: Fine-Tuning Complete ---")

# ------------ 6. FINAL EVALUATION ------------
print(f"\n--- Evaluating Final Model from {BEST_MODEL_PATH} ---")

# Sabse achha model load karein jo save hua tha
try:
    best_model = load_model(BEST_MODEL_PATH)
    
    test_loss, test_acc = best_model.evaluate(test_generator)
    print(f"\nFINAL Test Accuracy: {test_acc*100:.2f}%")
    print(f"FINAL Test Loss: {test_loss:.4f}")

    # Predictions
    predictions = best_model.predict(test_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    # Classification Report
    print("\n--- FINAL Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    # Confusion Matrix
    print("\n--- FINAL Confusion Matrix ---")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

except Exception as e:
    print(f"Error during final evaluation: {e}")
    print("It's possible training did not improve and no model was saved.")

print("\n--- Complete Process Finished ---")

--- Starting Complete Training & Fine-Tuning Process ---
Data directory: C:\JN\Aerial_Object_Classification\classification_dataset
Models directory: C:\JN\Aerial_Object_Classification\models
Best model will be saved to: C:\JN\Aerial_Object_Classification\models\final_finetuned_model.h5

Loading and preprocessing data...
Found 2662 images belonging to 2 classes.
Found 442 images belonging to 2 classes.
Found 215 images belonging to 2 classes.
Data loaded successfully.
Classes found: {'bird': 0, 'drone': 1}

Building model (EfficientNetB0)...

--- PHASE 1: Starting Feature Extraction (Training Classifier) ---
Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 287s 3s/step - accuracy: 0.5293 - loss: 0.6974 - val_accuracy: 0.5090 - val_loss: 0.6965
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5180 - loss: 0.6958 - val_accuracy: 0.4910 - val_loss: 0.6949
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 128s 2s/step - accuracy: 0.5207 - loss: 0.6952 - val_accuracy: 0.4910 - val_loss: 0.7077
Epoc

84/84 ━━━━━━━━━━━━━━━━━━━━ 1112s 11s/step - accuracy: 0.7002 - loss: 0.6038 - val_accuracy: 0.4140 - val_loss: 0.7016
Epoch 12/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.8240 - loss: 0.4837

84/84 ━━━━━━━━━━━━━━━━━━━━ 646s 8s/step - accuracy: 0.8403 - loss: 0.4602 - val_accuracy: 0.4163 - val_loss: 0.7131
Epoch 13/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.8650 - loss: 0.3888

84/84 ━━━━━━━━━━━━━━━━━━━━ 1076s 13s/step - accuracy: 0.8858 - loss: 0.3634 - val_accuracy: 0.5385 - val_loss: 0.6802
Epoch 14/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9020 - loss: 0.3163 

84/84 ━━━━━━━━━━━━━━━━━━━━ 1665s 20s/step - accuracy: 0.9023 - loss: 0.3033 - val_accuracy: 0.7443 - val_loss: 0.5601
Epoch 15/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9104 - loss: 0.2728

...

84/84 ━━━━━━━━━━━━━━━━━━━━ 1899s 23s/step - accuracy: 0.9643 - loss: 0.1136 - val_accuracy: 0.9706 - val_loss: 0.0972
Epoch 26/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 909s 10s/step - accuracy: 0.9703 - loss: 0.0981 - val_accuracy: 0.9683 - val_loss: 0.0920
Epoch 27/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9728 - loss: 0.0936

84/84 ━━━━━━━━━━━━━━━━━━━━ 753s 9s/step - accuracy: 0.9722 - loss: 0.0934 - val_accuracy: 0.9751 - val_loss: 0.0737
Epoch 28/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 795s 9s/step - accuracy: 0.9707 - loss: 0.0854 - val_accuracy: 0.9593 - val_loss: 0.1329
Epoch 29/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 681s 8s/step - accuracy: 0.9666 - loss: 0.0904 - val_accuracy: 0.9638 - val_loss: 0.1121
Epoch 30/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 669s 8s/step - accuracy: 0.9748 - loss: 0.0762 - val_accuracy: 0.9661 - val_loss: 0.0987

--- PHASE 2: Fine-Tuning Complete ---

--- Evaluating Final Model from C:\JN\Aerial_Object_Classification\models\final_finetuned_model.h5 ---


7/7 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.5628 - loss: 848.7739

FINAL Test Accuracy: 56.28%
FINAL Test Loss: 848.7739
7/7 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step

--- FINAL Classification Report ---


C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

              precision    recall  f1-score   support

        bird       0.56      1.00      0.72       121
       drone       0.00      0.00      0.00        94

    accuracy                           0.56       215
   macro avg       0.28      0.50      0.36       215
weighted avg       0.32      0.56      0.41       215


--- FINAL Confusion Matrix ---
[[121   0]
 [ 94   0]]

--- Complete Process Finished ---


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

print("--- FIX: Loading and Re-compiling the Best Model ---")

# --- 1. जरूरी वेरिएबल्स ---
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATA_DIR = r'C:\JN\Aerial_Object_Classification\classification_dataset'
MODELS_DIR = r'C:\JN\Aerial_Object_Classification\models'
BEST_MODEL_PATH = os.path.join(MODELS_DIR, 'final_finetuned_model.h5')

# --- 2. सिर्फ 'test_generator' को फिर से लोड करें ---
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'TEST'),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# --- 3. मॉडल लोड करें (यह वही 97% वाला मॉडल है) ---
print(f"Loading best model from {BEST_MODEL_PATH}...")
best_model = load_model(BEST_MODEL_PATH)

# --- 4. *** YAHAN ERROR FIX KIYA GAYA HAI *** ---
# हम मॉडल को इवैल्यूएट करने से पहले उसे Re-compile करते हैं।
# यह Keras/TensorFlow के 'Batch Normalization' लोडिंग बग को ठीक करता है।
print("Re-compiling model to fix Batch Norm layer states...")
best_model.compile(
    optimizer=Adam(learning_rate=1e-5), # लर्निंग रेट वही होना चाहिए जो फाइन-ट्यूनिंग में था
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --- 5. अब इवैल्यूएट करें (Evaluate) ---
print("Evaluating the correctly loaded model...")

test_loss, test_acc = best_model.evaluate(test_generator)
print(f"\n--- YAY! REAL FINAL Test Accuracy: {test_acc*100:.2f}% ---")
print(f"REAL FINAL Test Loss: {test_loss:.4f}")

# Predictions
predictions = best_model.predict(test_generator)
y_pred = (predictions > 0.5).astype(int)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification Report
print("\n--- REAL FINAL Classification Report ---")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
print("\n--- REAL FINAL Confusion Matrix ---")
cm = confusion_matrix(y_true, y_pred)
print(cm)

--- FIX: Loading and Re-compiling the Best Model ---
Found 215 images belonging to 2 classes.
Loading best model from C:\JN\Aerial_Object_Classification\models\final_finetuned_model.h5...


Re-compiling model to fix Batch Norm layer states...
Evaluating the correctly loaded model...
7/7 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5628 - loss: 848.7739

--- YAY! REAL FINAL Test Accuracy: 56.28% ---
REAL FINAL Test Loss: 848.7739
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step

--- REAL FINAL Classification Report ---
              precision    recall  f1-score   support

        bird       0.56      1.00      0.72       121
       drone       0.00      0.00      0.00        94

    accuracy                           0.56       215
   macro avg       0.28      0.50      0.36       215
weighted avg       0.32      0.56      0.41       215


--- REAL FINAL Confusion Matrix ---
[[121   0]
 [ 94   0]]


C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [ ]:
# ------------ IMPORTS ------------
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os

print("--- Starting Final Training Process (Saving to .keras format) ---")

# ------------ 1. SETUP PATHS & CREATE DIRECTORY ------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

DATA_DIR = r'C:\JN\Aerial_Object_Classification\classification_dataset'
MODELS_DIR = r'C:\JN\Aerial_Object_Classification\models'

# *** YAHAN FIX KIYA GAYA HAI ***
# Hum naye, sahi format (.keras) ka istemal karenge
BEST_MODEL_PATH = os.path.join(MODELS_DIR, 'final_model.keras')

os.makedirs(MODELS_DIR, exist_ok=True)
print(f"Data directory: {DATA_DIR}")
print(f"Best model will be saved to: {BEST_MODEL_PATH}")

# ------------ 2. DATA LOADING & PREPROCESSING ------------
print("\nLoading and preprocessing data...")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

try:
    train_generator = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TRAIN'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary'
    )
    validation_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'VALID'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    test_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, 'TEST'),
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False
    )
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"ERROR: Data directory not found. Please check the path: {e}")
    raise e

# ------------ 3. MODEL BUILDING ------------
print("\nBuilding model (EfficientNetB0)...")
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False 
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# ------------ 4. PHASE 1: FEATURE EXTRACTION TRAINING ------------
print("\n--- PHASE 1: Starting Feature Extraction ---")
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
PHASE_1_EPOCHS = 10
model.fit(
    train_generator,
    epochs=PHASE_1_EPOCHS,
    validation_data=validation_generator
)
print("\n--- PHASE 1: Complete ---")

# ------------ 5. PHASE 2: FINE-TUNING TRAINING ------------
print("\n--- PHASE 2: Starting Fine-Tuning ---")
base_model.trainable = True
unfreeze_layers = 30
for layer in base_model.layers[-unfreeze_layers:]:
    layer.trainable = True
print(f"Unfreezing last {unfreeze_layers} layers.")

model.compile(
    optimizer=Adam(learning_rate=1e-5), # Kam learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Callbacks (Filepath .keras format mein hai)
model_checkpoint = ModelCheckpoint(
    filepath=BEST_MODEL_PATH, # .keras format
    monitor='val_accuracy',
    save_best_only=True
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5 
)

PHASE_2_EPOCHS = 20
TOTAL_EPOCHS = PHASE_1_EPOCHS + PHASE_2_EPOCHS

model.fit(
    train_generator,
    epochs=TOTAL_EPOCHS,
    initial_epoch=PHASE_1_EPOCHS,
    validation_data=validation_generator,
    callbacks=[model_checkpoint, early_stopping]
)
print("\n--- PHASE 2: Fine-Tuning Complete ---")

# ------------ 6. FINAL EVALUATION ------------
print(f"\n--- Evaluating Final Model from {BEST_MODEL_PATH} ---")

try:
    # Nayi .keras file ko load karein
    best_model = load_model(BEST_MODEL_PATH)
    
    # Batch Norm bug ke liye safety measure (Re-compile)
    print("Re-compiling loaded model for safety...")
    best_model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print("Evaluating model...")
    test_loss, test_acc = best_model.evaluate(test_generator)
    
    print(f"\n--- REAL FINAL Test Accuracy: {test_acc*100:.2f}% ---")
    print(f"REAL FINAL Test Loss: {test_loss:.4f}")

    predictions = best_model.predict(test_generator)
    y_pred = (predictions > 0.5).astype(int)
    y_true = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    print("\n--- REAL FINAL Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=class_labels))
    print("\n--- REAL FINAL Confusion Matrix ---")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

except Exception as e:
    print(f"Error during final evaluation: {e}")

print("\n--- Complete Process Finished ---")

--- Starting Final Training Process (Saving to .keras format) ---
Data directory: C:\JN\Aerial_Object_Classification\classification_dataset
Best model will be saved to: C:\JN\Aerial_Object_Classification\models\final_model.keras

Loading and preprocessing data...
Found 2662 images belonging to 2 classes.
Found 442 images belonging to 2 classes.
Found 215 images belonging to 2 classes.
Data loaded successfully.

Building model (EfficientNetB0)...
